# Day 5 - LangGraph

In [ ]:
%pip install langchain langgraph langchain_community  -q

In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Literal
import operator

# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.3,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Define the state
class FinancialCoachState(TypedDict):
    messages: Annotated[list, operator.add]
    user_question: str
    question_category: str
    user_profile: dict
    advice: str
    action_plan: list
    follow_up_questions: list

# Node 1: Classify the financial question
def classify_question(state: FinancialCoachState) -> FinancialCoachState:
    """Classify the type of financial question"""
    question = state["user_question"]
    
    prompt = f"""Classify this financial question into ONE category:
- BUDGETING: Questions about spending, saving, expense tracking
- INVESTING: Questions about stocks, bonds, funds, portfolio
- DEBT: Questions about loans, credit cards, debt repayment
- RETIREMENT: Questions about 401k, IRA, pension, retirement planning
- GENERAL: General financial advice or multiple topics

Question: {question}

Respond with ONLY the category name."""
    
    response = llm.invoke(prompt)
    category = response.content.strip()
    
    state["question_category"] = category
    state["messages"].append(f"Classified as: {category}")
    return state

# Node 2: Extract user profile
def extract_user_profile(state: FinancialCoachState) -> FinancialCoachState:
    """Extract relevant user financial profile from question"""
    question = state["user_question"]
    
    prompt = f"""Extract financial profile information from this question.
Return a JSON-like summary with: age, income, savings, debts, goals.
Use 'unknown' for missing information.

Question: {question}

Format:
Age: [value or unknown]
Income: [value or unknown]
Savings: [value or unknown]
Debts: [value or unknown]
Goals: [brief description or unknown]"""
    
    response = llm.invoke(prompt)
    
    state["user_profile"] = {"raw": response.content}
    state["messages"].append("Profile extracted")
    return state

# Node 3: Provide budgeting advice
def provide_budgeting_advice(state: FinancialCoachState) -> FinancialCoachState:
    """Specialized advice for budgeting questions"""
    question = state["user_question"]
    profile = state["user_profile"].get("raw", "No profile")
    
    prompt = f"""As a financial coach, provide budgeting advice for this question.

User Profile:
{profile}

Question: {question}

Provide:
1. Specific budgeting recommendations
2. The 50/30/20 rule application if relevant
3. Tools or apps they could use
4. Common budgeting mistakes to avoid

Keep it practical and actionable."""
    
    response = llm.invoke(prompt)
    state["advice"] = response.content
    state["messages"].append("Budgeting advice provided")
    return state

# Node 4: Provide investing advice
def provide_investing_advice(state: FinancialCoachState) -> FinancialCoachState:
    """Specialized advice for investing questions"""
    question = state["user_question"]
    profile = state["user_profile"].get("raw", "No profile")
    
    prompt = f"""As a financial coach, provide investment advice for this question.

User Profile:
{profile}

Question: {question}

Provide:
1. Investment strategy based on age and risk tolerance
2. Asset allocation recommendations
3. Specific investment vehicles (index funds, ETFs, etc.)
4. Risk warnings and diversification advice
5. Disclaimer about consulting financial advisors

Keep it educational and balanced."""
    
    response = llm.invoke(prompt)
    state["advice"] = response.content
    state["messages"].append("Investment advice provided")
    return state

# Node 5: Provide debt management advice
def provide_debt_advice(state: FinancialCoachState) -> FinancialCoachState:
    """Specialized advice for debt management"""
    question = state["user_question"]
    profile = state["user_profile"].get("raw", "No profile")
    
    prompt = f"""As a financial coach, provide debt management advice for this question.

User Profile:
{profile}

Question: {question}

Provide:
1. Debt repayment strategy (avalanche vs snowball method)
2. Specific action steps prioritized by impact
3. Ways to negotiate with creditors
4. How to avoid accumulating more debt
5. Timeline expectations for becoming debt-free

Be encouraging and realistic."""
    
    response = llm.invoke(prompt)
    state["advice"] = response.content
    state["messages"].append("Debt management advice provided")
    return state

# Node 6: Provide retirement advice
def provide_retirement_advice(state: FinancialCoachState) -> FinancialCoachState:
    """Specialized advice for retirement planning"""
    question = state["user_question"]
    profile = state["user_profile"].get("raw", "No profile")
    
    prompt = f"""As a financial coach, provide retirement planning advice for this question.

User Profile:
{profile}

Question: {question}

Provide:
1. Retirement account options (401k, IRA, Roth IRA)
2. Contribution strategies and limits
3. Employer match optimization
4. Retirement savings milestones by age
5. Long-term planning considerations

Include specific numbers and timelines."""
    
    response = llm.invoke(prompt)
    state["advice"] = response.content
    state["messages"].append("Retirement advice provided")
    return state

# Node 7: Provide general advice
def provide_general_advice(state: FinancialCoachState) -> FinancialCoachState:
    """General financial advice"""
    question = state["user_question"]
    profile = state["user_profile"].get("raw", "No profile")
    
    prompt = f"""As a financial coach, provide comprehensive financial advice for this question.

User Profile:
{profile}

Question: {question}

Provide holistic financial guidance covering relevant aspects."""
    
    response = llm.invoke(prompt)
    state["advice"] = response.content
    state["messages"].append("General advice provided")
    return state

# Node 8: Create action plan
def create_action_plan(state: FinancialCoachState) -> FinancialCoachState:
    """Create a step-by-step action plan"""
    advice = state["advice"]
    
    prompt = f"""Based on this financial advice, create a numbered action plan with 3-5 specific steps.
Each step should be concrete and actionable.

Advice:
{advice}

Format as:
1. [First action step]
2. [Second action step]
3. [Third action step]
etc."""
    
    response = llm.invoke(prompt)
    state["action_plan"] = response.content.split('\n')
    state["messages"].append("Action plan created")
    return state

# Node 9: Generate follow-up questions
def generate_followup(state: FinancialCoachState) -> FinancialCoachState:
    """Generate relevant follow-up questions"""
    category = state["question_category"]
    advice = state["advice"]
    
    prompt = f"""Based on this financial advice in the {category} category, 
generate 2-3 relevant follow-up questions the user might want to ask next.

Advice given:
{advice}

Format as a simple numbered list."""
    
    response = llm.invoke(prompt)
    state["follow_up_questions"] = response.content.split('\n')
    state["messages"].append("Follow-up questions generated")
    return state

# Router function
def route_by_category(state: FinancialCoachState) -> str:
    """Route to appropriate advice node based on category"""
    category = state["question_category"]
    
    routing = {
        "BUDGETING": "budgeting",
        "INVESTING": "investing",
        "DEBT": "debt",
        "RETIREMENT": "retirement",
        "GENERAL": "general"
    }
    
    return routing.get(category, "general")

# Build the graph
workflow = StateGraph(FinancialCoachState)

# Add nodes
workflow.add_node("classify", classify_question)
workflow.add_node("extract_profile", extract_user_profile)
workflow.add_node("budgeting", provide_budgeting_advice)
workflow.add_node("investing", provide_investing_advice)
workflow.add_node("debt", provide_debt_advice)
workflow.add_node("retirement", provide_retirement_advice)
workflow.add_node("general", provide_general_advice)
workflow.add_node("action_plan", create_action_plan)
workflow.add_node("followup", generate_followup)

# Add edges
workflow.set_entry_point("classify")
workflow.add_edge("classify", "extract_profile")
workflow.add_conditional_edges(
    "extract_profile",
    route_by_category,
    {
        "budgeting": "budgeting",
        "investing": "investing",
        "debt": "debt",
        "retirement": "retirement",
        "general": "general"
    }
)
workflow.add_edge("budgeting", "action_plan")
workflow.add_edge("investing", "action_plan")
workflow.add_edge("debt", "action_plan")
workflow.add_edge("retirement", "action_plan")
workflow.add_edge("general", "action_plan")
workflow.add_edge("action_plan", "followup")
workflow.add_edge("followup", END)

# Compile the graph
financial_coach_app = workflow.compile()

# Example usage
print("=" * 70)
print("FINANCIAL COACHING AGENT - LangGraph Version")
print("=" * 70)

# Test Case 1: Investment Question
print("\n💰 TEST CASE 1: Investment Question")
print("-" * 70)

result1 = financial_coach_app.invoke({
    "messages": [],
    "user_question": "I'm 32 years old with $20,000 to invest. Should I put it in index funds or try stock picking?",
    "question_category": "",
    "user_profile": {},
    "advice": "",
    "action_plan": [],
    "follow_up_questions": []
})

print(f"\n📝 Question: {result1['user_question']}")
print(f"\n🏷️  Category: {result1['question_category']}")
print(f"\n👤 Profile:\n{result1['user_profile'].get('raw', 'N/A')}")
print(f"\n💡 Advice:\n{result1['advice']}")
print(f"\n✅ Action Plan:")
for step in result1['action_plan']:
    if step.strip():
        print(f"   {step}")
print(f"\n❓ Follow-up Questions:")
for question in result1['follow_up_questions']:
    if question.strip():
        print(f"   {question}")

# Test Case 2: Debt Management
print("\n" + "=" * 70)
print("💳 TEST CASE 2: Debt Management")
print("-" * 70)

result2 = financial_coach_app.invoke({
    "messages": [],
    "user_question": "I have $25,000 in student loans and $8,000 in credit card debt. What should I pay off first?",
    "question_category": "",
    "user_profile": {},
    "advice": "",
    "action_plan": [],
    "follow_up_questions": []
})

print(f"\n📝 Question: {result2['user_question']}")
print(f"\n🏷️  Category: {result2['question_category']}")
print(f"\n💡 Advice:\n{result2['advice']}")
print(f"\n✅ Action Plan:")
for step in result2['action_plan']:
    if step.strip():
        print(f"   {step}")

print("\n" + "=" * 70)
print("Session Complete! 🎉")
print("=" * 70)